In [1]:
!pip install pandas
!pip install nltk
!pip install textblob

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/pkgs/arc/python/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/pkgs/arc/python/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/pkgs/arc/python/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textblob import TextBlob
import string

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/alanlai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alanlai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alanlai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/alanlai/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
reviews_df = pd.read_json('data/yelp_academic_dataset_review.json', orient='record', lines=True)
business_df = pd.read_json('data/yelp_academic_dataset_business.json', orient='record', lines=True)

In [5]:
def get_year(df):
    tmp = df.copy()
    tmp['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')  
    tmp['year'] = pd.DatetimeIndex(df['date']).year
    return tmp

In [6]:
tmp = get_year(reviews_df)

In [7]:
tmp.year.value_counts()

2019    907284
2018    906362
2017    820048
2016    758882
2015    688415
2021    618189
2020    554557
2014    522275
2013    383950
2012    286570
2011    230813
2010    138587
2009     74387
2008     48226
2022     31665
2007     15363
2006      3853
2005       854
Name: year, dtype: int64

In [8]:
# Filter out reviews for restaurants not in PA 
business_list = business_df[business_df['state']=='PA']['business_id']
df = reviews_df[reviews_df['business_id'].isin(business_list)]

In [9]:
def split_time_to_feature(df):

    df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
    
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['month'] = pd.DatetimeIndex(df['date']).month
    df['day'] = pd.DatetimeIndex(df['date']).day
    df['hour'] = pd.DatetimeIndex(df['date']).hour
    df['min'] = pd.DatetimeIndex(df['date']).minute
    df['sec'] = pd.DatetimeIndex(df['date']).second
    
split_time_to_feature(df)

/tmp/ipykernel_641527/2456371630.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
/tmp/ipykernel_641527/2456371630.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = pd.DatetimeIndex(df['date']).year
/tmp/ipykernel_641527/2456371630.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [10]:
def feature_engineering(df):
    # Creating a new column in the dataset for the length of the reviews
    df['length_of_reviews'] = df['text'].apply(len)

    # Creating a new column in the dataset for the number of words in the reviews
    df['num_of_words'] = df['text'].apply(lambda str:len(nltk.word_tokenize(str)))

    # Creating a new column in the dataset for the number of sentences in the reviews
    df['num_of_sentences'] = df['text'].apply(lambda paragraph:len(nltk.sent_tokenize(paragraph)))

    df['capital_words'] = df['text'].apply(lambda sen:len(re.findall(r'\b[A-Z]+\b', sen)))
    df['capital_words_ratio'] = df['capital_words']/df['num_of_words']
    df.drop(columns='capital_words', inplace = True)
    
feature_engineering(df)

/tmp/ipykernel_641527/2475708638.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['length_of_reviews'] = df['text'].apply(len)
/tmp/ipykernel_641527/2475708638.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num_of_words'] = df['text'].apply(lambda str:len(nltk.word_tokenize(str)))
/tmp/ipykernel_641527/2475708638.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [11]:
# Cleaning the reviews - stemed, remove stopwords and punctuation
def clean_text(text):

    ## Remove puncuation
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    ## Remove stop words
    nostop = [word for word in nopunc.split() if word.lower() not in stopwords.words('english') and len(word) >= 3]
    text = ' '.join(nostop)
   
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)    
    
    ps = PorterStemmer()
    text = ps.stem(text)

    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text)

    return text

df['cleaned_text'] = df['text'].apply(clean_text)

/tmp/ipykernel_641527/2254188807.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text'] = df['text'].apply(clean_text)


In [12]:
# Sentiment analysis using textblob 
df['sentiment_polarity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).polarity)
df['sentiment_subjectivity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).subjectivity)

/tmp/ipykernel_641527/632973997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_polarity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).polarity)
/tmp/ipykernel_641527/632973997.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_subjectivity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).subjectivity)


In [13]:
df.to_csv('data/cleaned_review_data.csv')